In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
data_cleaned = pd.read_excel('./data_engineered.xlsx')
data_cleaned.head()

,id,keyword,location,text,target,text_nolink,hashtags,text_nomentions,extracted_locations,noPunctuations,noStopwordsTokenized,lemmatized,text_cleaned,all_locations,extra_keywords,all_keywords
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,Our Deeds are the Reason of this #earthquake M...,['earthquake'],Our Deeds are the Reason of this #earthquake M...,[],Our Deeds are the Reason of this earthquake Ma...,"['Our', 'Deeds', 'Reason', 'earthquake', 'May'...","['Our', 'Deeds', 'Reason', 'earthquake', 'May'...",our deeds reason earthquake may allah forgive u,NaN,['earthquake'],earthquake
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,Forest fire near La Ronge Sask. Canada,[],Forest fire near La Ronge Sask. Canada,['Canada'],Forest fire near La Ronge Sask Canada,"['Forest', 'fire', 'near', 'La', 'Ronge', 'Sas...","['Forest', 'fire', 'near', 'La', 'Ronge', 'Sas...",forest fire near la ronge sask canada,Canada,[],NaN
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,All residents asked to 'shelter in place' are ...,[],All residents asked to 'shelter in place' are ...,[],All residents asked to shelter in place are be...,"['All', 'residents', 'asked', 'shelter', 'plac...","['All', 'resident', 'asked', 'shelter', 'place...",all resident asked shelter place notified offi...,NaN,[],NaN
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,"13,000 people receive #wildfires evacuation or...",['wildfires'],"13,000 people receive #wildfires evacuation or...",['California'],13000 people receive wildfires evacuation orde...,"['13000', 'people', 'receive', 'wildfires', 'e...","['13000', 'people', 'receive', 'wildfire', 'ev...",13000 people receive wildfire evacuation order...,California,['wildfires'],wildfires
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,Just got sent this photo from Ruby #Alaska as ...,"['Alaska', 'wildfires']",Just got sent this photo from Ruby #Alaska as ...,"['Ruby', 'Alaska']",Just got sent this photo from Ruby Alaska as s...,"['Just', 'got', 'sent', 'photo', 'Ruby', 'Alas...","['Just', 'got', 'sent', 'photo', 'Ruby', 'Alas...",just got sent photo ruby alaska smoke wildfire...,Ruby Alaska,['wildfires'],wildfires


In [3]:
df = data_cleaned[['text_cleaned', 'all_locations', 'all_keywords', 'target']]

In [4]:
df.head()

,text_cleaned,all_locations,all_keywords,target
0,our deeds reason earthquake may allah forgive u,NaN,earthquake,1
1,forest fire near la ronge sask canada,Canada,NaN,1
2,all resident asked shelter place notified offi...,NaN,NaN,1
3,13000 people receive wildfire evacuation order...,California,wildfires,1
4,just got sent photo ruby alaska smoke wildfire...,Ruby Alaska,wildfires,1


In [5]:
df['text_length'] = df['text_cleaned'].apply(lambda x:len(x))
df['word_count'] = df['text_cleaned'].apply(lambda x:len(x.split()))

In [6]:
df.isna().sum()

text_cleaned        0
all_locations    2110
all_keywords       41
target              0
text_length         0
word_count          0
dtype: int64

In [7]:
df.fillna("NA", inplace=True)

In [8]:
df.isna().sum()

text_cleaned     0
all_locations    0
all_keywords     0
target           0
text_length      0
word_count       0
dtype: int64

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.compose import make_column_transformer
cv = CountVectorizer()

X = cv.fit_transform(df['text_cleaned']).toarray()

In [10]:
X = np.c_[X, df['text_length'], df['word_count']]

In [11]:
Y = df['target']

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [13]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix

classifier1 = GaussianNB()
classifier1.fit(X_train, Y_train)

Y_pred = classifier1.predict(X_test)

print(confusion_matrix(Y_test, Y_pred))
print(accuracy_score(Y_test, Y_pred))

[[630 688]
 [177 789]]
0.6212784588441331


In [14]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

treeClassifier1 = DecisionTreeClassifier()
treeClassifier1.fit(X_train, Y_train)

Y_pred = treeClassifier1.predict(X_test)

print(confusion_matrix(Y_test, Y_pred))
print(accuracy_score(Y_test, Y_pred))

[[1048  270]
 [ 329  637]]
0.7377408056042032


In [15]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()
clf.fit(X_train, Y_train)

Y_pred = clf.predict(X_test)

print(confusion_matrix(Y_test, Y_pred))
print(accuracy_score(Y_test, Y_pred))

[[1205  113]
 [ 375  591]]
0.7863397548161121


In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

lr = LogisticRegression(penalty='l2', C=1, max_iter=7600, random_state=42)
lr.fit(X_train, Y_train)

Y_pred = lr.predict(X_test)

print(confusion_matrix(Y_test, Y_pred))
print(accuracy_score(Y_test, Y_pred))

[[1132  186]
 [ 272  694]]
0.7994746059544658


In [17]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score, confusion_matrix

bnb = BernoulliNB()
bnb.fit(X_train, Y_train)

Y_pred = bnb.predict(X_test)

print(confusion_matrix(Y_test, Y_pred))
print(accuracy_score(Y_test, Y_pred))

[[1204  114]
 [ 337  629]]
0.8025394045534151


In [19]:
# import lazypredict
# from lazypredict.Supervised import LazyClassifier

# clf = LazyClassifier(verbose=True, ignore_warnings=True)
# models, predictions = clf.fit(X_train, X_test, Y_train, Y_test)
# models

# Testing the data on the given `test.csv`

In [20]:
data_test = pd.read_csv('../nlp-getting-started/test.csv')
data_test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [21]:
from CleanData import CleanData

cleanDataobj = CleanData(data_test)

In [22]:
TestData_cleaned = cleanDataobj.clean()
TestData_cleaned.head()

,id,keyword,location,text,text_nolink,hashtags,text_nomentions,extracted_locations,noPunctuations,noStopwordsTokenized,lemmatized,text_cleaned,all_locations,extra_keywords,all_keywords
0,0,NA,NA,Just happened a terrible car crash,Just happened a terrible car crash,[],Just happened a terrible car crash,[],Just happened a terrible car crash,"[Just, happened, terrible, car, crash]","[Just, happened, terrible, car, crash]",just happened terrible car crash,,[],
1,2,NA,NA,"Heard about #earthquake is different cities, s...","Heard about #earthquake is different cities, s...",[earthquake],"Heard about #earthquake is different cities, s...",[],Heard about earthquake is different cities sta...,"[Heard, earthquake, different, cities, stay, s...","[Heard, earthquake, different, city, stay, saf...",heard earthquake different city stay safe ever...,,[earthquake],earthquake
2,3,NA,NA,"there is a forest fire at spot pond, geese are...","there is a forest fire at spot pond, geese are...",[],"there is a forest fire at spot pond, geese are...",[],there is a forest fire at spot pond geese are ...,"[forest, fire, spot, pond, geese, fleeing, acr...","[forest, fire, spot, pond, goose, fleeing, acr...",forest fire spot pond goose fleeing across str...,,[],
3,9,NA,NA,Apocalypse lighting. #Spokane #wildfires,Apocalypse lighting. #Spokane #wildfires,"[Spokane, wildfires]",Apocalypse lighting. #Spokane #wildfires,[],Apocalypse lighting Spokane wildfires,"[Apocalypse, lighting, Spokane, wildfires]","[Apocalypse, lighting, Spokane, wildfire]",apocalypse lighting spokane wildfire,,"[Spokane, wildfires]",Spokane wildfires
4,11,NA,NA,Typhoon Soudelor kills 28 in China and Taiwan,Typhoon Soudelor kills 28 in China and Taiwan,[],Typhoon Soudelor kills 28 in China and Taiwan,"[China, Taiwan]",Typhoon Soudelor kills 28 in China and Taiwan,"[Typhoon, Soudelor, kills, 28, China, Taiwan]","[Typhoon, Soudelor, kill, 28, China, Taiwan]",typhoon soudelor kill 28 china taiwan,Taiwan China,[],


In [23]:
id_col = TestData_cleaned['id']

In [24]:
TestData = TestData_cleaned[['text_cleaned', 'all_locations', 'all_keywords']]

In [25]:
TestData['text_length'] = TestData['text_cleaned'].apply(lambda x:len(x))
TestData['word_count'] = TestData['text_cleaned'].apply(lambda x:len(x.split()))

In [26]:
TestData.head()

,text_cleaned,all_locations,all_keywords,text_length,word_count
0,just happened terrible car crash,,,32,5
1,heard earthquake different city stay safe ever...,,earthquake,50,7
2,forest fire spot pond goose fleeing across str...,,,63,11
3,apocalypse lighting spokane wildfire,,Spokane wildfires,36,4
4,typhoon soudelor kill 28 china taiwan,Taiwan China,,37,6


In [27]:
TestData.fillna("NA", inplace=True)

In [28]:
Test_X = cv.transform(TestData['text_cleaned']).toarray()
Test_X = np.c_[Test_X, TestData['text_length'], TestData['word_count']]

In [29]:
Test_X

array([[ 0,  0,  0, ...,  0, 32,  5],
       [ 0,  0,  0, ...,  0, 50,  7],
       [ 0,  0,  0, ...,  0, 63, 11],
       ...,
       [ 0,  0,  0, ...,  0, 29,  4],
       [ 0,  0,  0, ...,  0, 39,  6],
       [ 0,  0,  0, ...,  0, 57,  6]], dtype=int64)

In [30]:
# predictionvalues = []
# for el in df_test:
# Y_pred_test = bnb.predict(Test_X)
Y_pred_test = lr.predict(Test_X)
print(Y_pred_test)

[1 1 1 ... 1 1 1]


In [31]:
submissionArray = np.c_[id_col, Y_pred_test]

In [32]:
submission = pd.DataFrame(submissionArray, columns=['id','target'])

In [33]:
submission.head()

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1


In [34]:
submission.to_csv('../../Natural Language Processing with Disaster Tweets/submission2.csv', index=False)